In [142]:
BACKEND_URL = 'https://migdar-internal-search.odata.org.il/__search/%40and%20%40attr%201%3DCode-language%20heb%20%40attr%201%3D1017%20%22abracadbra%22/%D7%91%D7%93%D7%99%D7%A7%D7%94%20%D7%90%D7%97%D7%AA%20%D7%A9%D7%AA%D7%99%D7%99%D7%9D%20%D7%A9%D7%9C%D7%95%D7%A9/heb/undefined'
print(BACKEND_URL)

https://migdar-internal-search.odata.org.il/__search/%40and%20%40attr%201%3DCode-language%20heb%20%40attr%201%3D1017%20%22abracadbra%22/%D7%91%D7%93%D7%99%D7%A7%D7%94%20%D7%90%D7%97%D7%AA%20%D7%A9%D7%AA%D7%99%D7%99%D7%9D%20%D7%A9%D7%9C%D7%95%D7%A9/heb/undefined


In [140]:
import requests
from IPython.core.display import HTML

res = requests.get(BACKEND_URL)
display(HTML(res.text))

In [13]:
# DELETE THE CREDENTIALS AFTER RUNNING!
AUTH = ('', '')

session = requests.session()
session.auth = AUTH

In [143]:

res = session.get(BACKEND_URL)
display(HTML(res.text))

## Start local backend server

In [2]:
%%bash
supervisord -c supervisord-development.conf
supervisorctl -c supervisord-development.conf status

search-app                       STARTING  


In [6]:
%%bash
supervisorctl -c supervisord-development.conf reload

Restarted supervisord


In [8]:
%%bash
supervisorctl -c supervisord-development.conf status

search-app                       RUNNING   pid 25284, uptime 0:00:07


In [9]:
%%bash
supervisorctl -c supervisord-development.conf tail search-app

 * Running on http://0.0.0.0:5050/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 126-776-375



In [14]:
LOCAL_BACKEND_URL = BACKEND_URL.replace('https://migdar-internal-search.odata.org.il', 'http://localhost:5050')
print(LOCAL_BACKEND_URL)

http://localhost:5050/search/%40and%20%40attr%201%3DCode-language%20heb%20%40attr%201%3D1017%20%22%D7%91%D7%93%D7%99%D7%A7%D7%94%20%D7%90%D7%97%D7%AA%20%D7%A9%D7%AA%D7%99%D7%99%D7%9D%20%D7%A9%D7%9C%D7%95%D7%A9%22/%D7%91%D7%93%D7%99%D7%A7%D7%94%20%D7%90%D7%97%D7%AA%20%D7%A9%D7%AA%D7%99%D7%99%D7%9D%20%D7%A9%D7%9C%D7%95%D7%A9/heb/undefined


In [138]:
from urllib.parse import quote

SEARCH_TEXT = 'abracadabra'
SEARCH_QUERY = f'@and @attr 1=Code-language heb @attr 1=1017 "{SEARCH_TEXT}"'
SEARCH_LANG = 'heb'
SEARCH_NO_CACHE = 'False'
SEARCH_NO_CACHE_ARG = 'true' if SEARCH_NO_CACHE else 'undefined'

LOCAL_BACKEND_URL = 'http://localhost:5050/__search/' + quote(f'{SEARCH_QUERY}/{SEARCH_TEXT}/{SEARCH_LANG}/{SEARCH_NO_CACHE_ARG}')
print(LOCAL_BACKEND_URL)

res = session.get(LOCAL_BACKEND_URL)
display(HTML(res.text))

http://localhost:5050/__search/%40and%20%40attr%201%3DCode-language%20heb%20%40attr%201%3D1017%20%22abracadabra%22/abracadabra/heb/true


## Calling the ULI server directly

### Using Docker

In [35]:
%%bash
DB_NAME="ULI02"
SEARCH_QUERY='@and @attr 1=Code-language heb @attr 1=1017 "abracadabra"'
QUERY_TYPE='PQF'
docker run -e "QUERY_TYPE=${QUERY_TYPE}" --entrypoint python2 uumpa/hasadna-nli-z3950 nli-z3950.py2 "${DB_NAME}" "${SEARCH_QUERY}"


[
]


Getting page 1
PQF = @and @attr 1=Code-language heb @attr 1=1017 "abracadabra"Got 0 records, stopping


### Using Python

The z3950 library only supports Python 2 with some specific (old) dependencies

Create a conda environment:

In [ ]:
%%bash
conda create -y -n nli-z3950-py2 'python<3'

In [118]:
%%bash
source $(dirname `which conda`)/activate nli-z3950-py2 &&\
pip install -e mollyZ3950-2.04-molly1 &&\
pip install pymarc

Obtaining file:///home/ori/nli-z3950/mollyZ3950-2.04-molly1
  Found existing installation: mollyZ3950 2.04-molly1
    Uninstalling mollyZ3950-2.04-molly1:
      Successfully uninstalled mollyZ3950-2.04-molly1
  Running setup.py develop for mollyZ3950
  Using cached https://files.pythonhosted.org/packages/2b/71/dd47f6bb599265317249503781302a74a7768474ccab2ac0b1ea3c51e076/pymarc-3.1.10.tar.gz
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Running setup.py bdist_wheel for pymarc: started
  Running setup.py bdist_wheel for pymarc: finished with status 'done'
  Stored in directory: /home/ori/.cache/pip/wheels/62/fc/4a/81f8b8e62a0931774fe8efb87616bcbfba879d4fe1c6ecc978
Successfully built pymarc


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Run a query

In [136]:
%%bash
DB_NAME="ULI02"
SEARCH_QUERY='@and @attr 1=Code-language heb @attr 1=1017 "abracadabra"'
QUERY_TYPE='PQF'

source $(dirname `which conda`)/activate nli-z3950-py2

QUERY_TYPE=${QUERY_TYPE} python2 nli-z3950.py2 "${DB_NAME}" "${SEARCH_QUERY}"


[
]


Getting page 1
PQF = @and @attr 1=Code-language heb @attr 1=1017 "abracadabra"Got 0 records, stopping
